<a href="https://colab.research.google.com/github/bye23mj/suanLee/blob/main/_13_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C(Recommender_System).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

간단한 surprise 실습

In [ ]:
import pandas as pd

from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=True)
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [ ]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=False)

{'fit_time': (5.463627099990845,
  5.3785083293914795,
  5.364475965499878,
  5.328824043273926,
  5.642615795135498),
 'test_mae': array([0.74351165, 0.73443449, 0.74521587, 0.73944029, 0.73349898]),
 'test_rmse': array([0.94265981, 0.93220562, 0.94464321, 0.93930846, 0.93079906]),
 'test_time': (0.27910876274108887,
  0.19211316108703613,
  0.2947657108306885,
  0.1702864170074463,
  0.1799008846282959)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [ ]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=True)
raw_data = np.array(data.raw_ratings, dtype=int)

raw_data[1,:]

array([      186,       302,         3, 891717742])

'/root/.surprise_data/ml-100k/ml-100k/u.data'

In [ ]:
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

raw_data[1,:]

array([      185,       301,         3, 891717742])

In [ ]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users + 1, n_movies + 1)

shape

(943, 1682)

In [ ]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = 1.


In [ ]:
my_id, my_vector = 1, adj_matrix[1]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    similarity = np.dot(my_vector, user_vector)
    if similarity > best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match : 168, Best Match ID: 12


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[1, 2, 7, 14, 16, 21, 26, 27, 28, 31, 34, 35, 37, 38, 39, 41, 44, 45, 47, 48, 50, 53, 54, 57, 58, 59, 61, 63, 65, 66, 67, 68, 71, 72, 73, 81, 82, 85, 86, 87, 90, 91, 94, 96, 97, 98, 107, 108, 109, 115, 116, 117, 120, 123, 127, 131, 134, 136, 137, 138, 140, 142, 143, 144, 146, 149, 151, 153, 154, 156, 157, 158, 159, 160, 161, 163, 164, 165, 166, 167, 169, 171, 172, 173, 174, 175, 177, 178, 179, 181, 183, 184, 185, 186, 189, 190, 192, 193, 195, 196, 197, 198, 201, 203, 204, 205, 206, 208, 210, 216, 217, 219, 221, 224, 226, 228, 229, 231, 232, 233, 234, 235, 237, 239, 242, 259, 261, 262, 264, 270, 317, 318, 319, 325, 326, 327, 328, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 342, 343, 345, 346, 347, 348, 349, 350, 351, 352, 353, 356, 357, 359, 362, 364, 366, 369, 370, 371, 373, 376, 378, 380, 381, 384, 386, 390, 391, 392, 393, 395, 402, 403, 408, 411, 412, 413, 416, 418, 419, 420, 429, 430, 432, 434, 435, 436, 437, 438, 439, 440, 441, 442, 444, 447, 448, 449, 451, 452, 453, 460

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector-user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match : 14.832396974191326, Best Match ID: 737


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

* ex) x = [-4 6 -2] , ||x|| or |x| = 12

In [ ]:
def compute_cos_similarity(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1,v2)
  return dot / (norm1 * norm2)

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match : 0.5278586163659506, Best Match ID: 915


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [ ]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match : 0.569065731527988, Best Match ID: 915


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [ ]:
from surprise import KNNBaseline, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)

* KNN을 사용한 협업 필터링

In [ ]:
model = KNNBaseline()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9290  0.9348  0.9316  0.9329  0.9222  0.9301  0.0044  
MAE (testset)     0.7301  0.7359  0.7325  0.7363  0.7278  0.7325  0.0033  
Fit time          0.46    0.62    1.06    1.04    0.68    0.77    0.24    
Test time         8.73    10.38   9.54    7.48    5.16    8.26    1.82    


{'fit_time': (0.45621609687805176,
  0.6189601421356201,
  1.05904221534729,
  1.043144702911377,
  0.6757817268371582),
 'test_mae': array([0.73010315, 0.73588486, 0.7325062 , 0.73633139, 0.72777424]),
 'test_rmse': array([0.92898036, 0.93480745, 0.93163265, 0.93287085, 0.92220233]),
 'test_time': (8.73091721534729,
  10.382155418395996,
  9.536654472351074,
  7.484034299850464,
  5.160220623016357)}

* SVD를 사용한 협업 필터링

In [ ]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9339  0.9306  0.9341  0.9426  0.9416  0.9365  0.0047  
MAE (testset)     0.7362  0.7332  0.7374  0.7410  0.7454  0.7386  0.0042  
Fit time          12.22   16.05   16.33   13.30   9.49    13.48   2.54    
Test time         1.22    0.61    0.45    0.31    0.17    0.55    0.36    


{'fit_time': (12.222621440887451,
  16.05112051963806,
  16.32511878013611,
  13.302451372146606,
  9.49233341217041),
 'test_mae': array([0.73622001, 0.73321901, 0.73742521, 0.74099189, 0.7453738 ]),
 'test_rmse': array([0.93388172, 0.93055973, 0.93407128, 0.9425824 , 0.94159474]),
 'test_time': (1.2196006774902344,
  0.6128401756286621,
  0.4463343620300293,
  0.3094956874847412,
  0.17447662353515625)}

* NMF를 사용한 협업 필터링

In [ ]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9585  0.9662  0.9638  0.9630  0.9584  0.9620  0.0031  
MAE (testset)     0.7534  0.7604  0.7580  0.7587  0.7541  0.7569  0.0027  
Fit time          13.32   16.58   16.23   13.67   9.30    13.82   2.61    
Test time         0.62    0.52    0.39    0.27    0.14    0.39    0.17    


{'fit_time': (13.321378946304321,
  16.57982611656189,
  16.225108861923218,
  13.669403314590454,
  9.29880404472351),
 'test_mae': array([0.75335034, 0.76038302, 0.75795303, 0.75868732, 0.75406901]),
 'test_rmse': array([0.95846466, 0.96622564, 0.96378445, 0.96304619, 0.95839184]),
 'test_time': (0.6160285472869873,
  0.5205094814300537,
  0.39405035972595215,
  0.2695930004119873,
  0.1428661346435547)}

* SVD++를 사용한 협업 필터링

In [ ]:
model = SVDpp()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9160  0.9163  0.9307  0.9137  0.9251  0.9203  0.0065  
MAE (testset)     0.7204  0.7182  0.7306  0.7171  0.7258  0.7224  0.0051  
Fit time          655.67  669.20  668.12  663.80  188.97  569.15  190.15  
Test time         12.40   12.57   11.73   10.77   3.11    10.12   3.56    


{'fit_time': (655.6677641868591,
  669.2000257968903,
  668.1196691989899,
  663.8001589775085,
  188.96682000160217),
 'test_mae': array([0.72038457, 0.71816604, 0.73064315, 0.71712502, 0.72578963]),
 'test_rmse': array([0.91595705, 0.9162503 , 0.93067186, 0.9137343 , 0.9250812 ]),
 'test_time': (12.399771451950073,
  12.573261260986328,
  11.72516918182373,
  10.774755239486694,
  3.1087372303009033)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [ ]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=True)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [ ]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users + 1, n_movies + 1)

In [ ]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

In [ ]:
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


In [ ]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [ ]:
np.matmul(np.matmul(U,S), V)

array([[ 4.75601227e-09, -1.64730293e-02, -9.07018107e-09, ...,
         1.75607681e-01,  1.55744664e-01,  1.35800104e-01],
       [ 3.13574954e-09, -1.08611090e-02, -5.98021324e-09, ...,
         1.05961142e-01,  9.28649235e-02,  7.97149411e-02],
       [ 9.37692381e-10, -3.24548205e-03, -1.78698257e-09, ...,
         4.32072774e-01,  4.28159403e-01,  4.24229967e-01],
       ...,
       [ 1.09335092e+02, -3.78715133e+08, -2.08523615e+02, ...,
         6.78018932e+08,  2.21367969e+08, -2.37157703e+08],
       [-1.05936766e+02,  3.66946556e+08,  2.02043748e+02, ...,
        -2.21874128e+08,  2.20586401e+08,  6.64863381e+08],
       [ 2.37271034e+02,  1.32829198e+05,  2.07567546e+00, ...,
         1.40215065e+14,  1.40215065e+14,  1.40215065e+14]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [ ]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match : 0.9999999999999648, Best Match ID: 590


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[284, 285, 292, 299, 303, 305, 317, 318, 321, 327, 331, 339, 341, 343, 345, 347, 353, 355, 356, 360, 389, 427, 446, 450, 456, 465, 497, 507, 510, 513, 515, 516, 522, 538, 548, 556, 570, 571, 579, 587, 614, 654, 661, 664, 708, 709, 711, 724, 731, 739, 776, 781, 791, 855, 865, 887, 920, 922, 933, 949, 953, 955, 965, 998, 1005, 1008, 1016, 1027, 1040, 1051, 1052, 1098, 1110, 1119, 1124, 1184, 1196, 1480, 1509, 1523, 1524, 1530, 1541, 1546, 1550, 1555, 1558, 1577, 1586, 1590, 1591, 1592, 1599, 1600, 1602, 1607, 1610, 1612, 1613, 1614, 1615, 1616, 1622, 1623, 1628, 1631, 1633, 1637, 1638, 1640, 1642, 1643, 1645, 1654, 1657, 1660, 1669]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [ ]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match : 0.9999999999992878, Best Match ID: 605


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9 :
    recommend_list.append(i)
print(recommend_list)

[0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 28, 30, 32, 33, 35, 36, 37, 40, 41, 42, 43, 44, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 87, 88, 89, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 104, 105, 106, 107, 108, 116, 118, 119, 120, 121, 123, 124, 126, 127, 129, 130, 132, 133, 134, 135, 136, 137, 139, 140, 141, 143, 144, 147, 148, 149, 150, 152, 154, 155, 156, 157, 159, 160, 161, 162, 166, 167, 168, 173, 174, 175, 176, 177, 178, 180, 181, 183, 184, 188, 190, 191, 192, 193, 196, 197, 198, 199, 200, 201, 202, 203, 204, 208, 209, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 226, 229, 230, 231, 233, 234, 235, 236, 239, 240, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 258, 260, 261, 262, 264, 265, 266, 267, 269, 270, 273, 274, 275, 276, 278, 279, 281, 282, 283, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295,

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [ ]:
adj_matrix

In [ ]:
A, B, iter = non_negative_factorization(adj_matrix, n_components=2)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [ ]:
np.matmul(A, B)

array([[2.76526164e-09, 5.39422325e-01, 2.99519834e-07, ...,
        1.26857187e-02, 1.26857187e-02, 6.62069536e-01],
       [7.65431272e-10, 1.49298951e-01, 8.28997897e-08, ...,
        4.74343123e-02, 4.74343123e-02, 2.27167924e-01],
       [5.36128403e-10, 1.04453187e-01, 5.79987199e-08, ...,
        3.96394813e-01, 3.96394813e-01, 5.22140831e-01],
       ...,
       [3.71328363e-04, 0.00000000e+00, 3.12611657e-06, ...,
        2.19651283e+08, 2.19651283e+08, 2.19651283e+08],
       [3.76537651e+00, 7.34521419e+08, 4.07850626e+02, ...,
        0.00000000e+00, 0.00000000e+00, 8.84253952e+08],
       [2.37038640e+02, 9.60913837e+03, 2.00090160e+00, ...,
        1.40215065e+14, 1.40215065e+14, 1.40215065e+14]])

* 사용자 기반 추천

In [ ]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match : 0.9999999999999648, Best Match ID: 590


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[284, 285, 292, 299, 303, 305, 317, 318, 321, 327, 331, 339, 341, 343, 345, 347, 353, 355, 356, 360, 389, 427, 446, 450, 456, 465, 497, 507, 510, 513, 515, 516, 522, 538, 548, 556, 570, 571, 579, 587, 614, 654, 661, 664, 708, 709, 711, 724, 731, 739, 776, 781, 791, 855, 865, 887, 920, 922, 933, 949, 953, 955, 965, 998, 1005, 1008, 1016, 1027, 1040, 1051, 1052, 1098, 1110, 1119, 1124, 1184, 1196, 1480, 1509, 1523, 1524, 1530, 1541, 1546, 1550, 1555, 1558, 1577, 1586, 1590, 1591, 1592, 1599, 1600, 1602, 1607, 1610, 1612, 1613, 1614, 1615, 1616, 1622, 1623, 1628, 1631, 1633, 1637, 1638, 1640, 1642, 1643, 1645, 1654, 1657, 1660, 1669]


* 항목 기반 추천

In [ ]:
my_id, my_vector = 0, B.T[0]  # 기준 항목이 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1, []

for item_id, item_vector in enumerate(V.T):
  if my_id != item_id:
    cos_similarity = compute_cos_similarity(my_vector, item_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = item_id
      best_match_vector = item_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9975211362823283, Best Match ID: 124


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 28, 30, 32, 33, 35, 36, 37, 40, 41, 42, 43, 44, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 87, 88, 89, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 104, 105, 106, 107, 108, 116, 118, 119, 120, 121, 123, 124, 126, 127, 129, 130, 132, 133, 134, 135, 136, 137, 139, 140, 141, 143, 144, 147, 148, 149, 150, 152, 154, 155, 156, 157, 159, 160, 161, 162, 166, 167, 168, 173, 174, 175, 176, 177, 178, 180, 181, 183, 184, 188, 190, 191, 192, 193, 196, 197, 198, 199, 200, 201, 202, 203, 204, 208, 209, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 226, 229, 230, 231, 233, 234, 235, 236, 239, 240, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 258, 260, 261, 262, 264, 265, 266, 267, 269, 270, 273, 274, 275, 276, 278, 279, 281, 282, 283, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295,

### 인터넷 실습

In [ ]:
from surprise import Dataset
from surprise.model_selection import train_test_split

# 내장 데이터인 무비렌즈 데이터 로드하고 학습/테스트 데이터로 분리
data = Dataset.load_builtin('ml-100k')
train, test = train_test_split(data, test_size=0.25, random_state=42)

import pandas as pd
import os
os.chdir('/Users/younghun/Desktop/gitrepo/data/ml-latest-small')
ratings = pd.read_csv('ratings.csv')
# Surprise 모듈에서 csv파일을 읽어오도록 포맷을 변경해주어야 하기 위해서 따로 저장
# 이 때, index값과 Header(칼럼명)값들 없애주면서 저장시키기
ratings.to_csv('ratings_surprise.csv', index=False, header=False)